# Plot Parameter space of all measurements

In [9]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import xarray as xr 
import scipy.stats as stat
import collections
import sys
import os
import glob
import re
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.core.colours as col
import flopter.core.ivdata as iv
import flopter.core.lputils as lp
import flopter.magnum.database as ut
import flopter.magnum.utils as mgut
import flopter.core.fitters as fts
import flopter.core.constants as c

In [2]:
# Create analysed dataset metadata 

path_to_datasets = '/home/jleland/data/external/magnum/'
# path_to_datasets = '/home/jleland/data/externy/magnum/'
# path_to_analysed_datasets = 'analysed_2'
# path_to_analysed_datasets = 'analysed_3'
# path_to_analysed_datasets = 'phobos_test'
# path_to_analysed_datasets = 'test'
# path_to_analysed_datasets = 'analysed_3_downsampled'
# path_to_analysed_datasets = 'analysed_4'
path_to_analysed_datasets = 'analysed_4_downsampled'
# path_to_analysed_datasets = 'analysed_5_downsampled'
os.chdir(path_to_datasets)

In [3]:
magnum_probes = lp.MagnumProbes()
print(magnum_probes.probe_position.items())

dict_items([('l', 6), ('s', -4), ('b', -14), ('r', -24)])


## Load adc file metadata

In [4]:
os.chdir('/home/jleland/data/external/magnum/')
# os.chdir('/home/jleland/data/externy/magnum/')
meta_data_ds = xr.open_dataset('all_meta_data.nc')
# print(meta_data_ds)

---
## Do the plotting

In [5]:
meta_data_ds

<xarray.Dataset>
Dimensions:                 (shot_number: 523, ts_radial_pos: 46)
Coordinates:
  * shot_number             (shot_number) int32 0 1 2 3 4 ... 519 520 521 522
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
Data variables:
    adc_filename            (shot_number) object ...
    ts_density              (shot_number, ts_radial_pos) float64 ...
    ts_temperature          (shot_number, ts_radial_pos) float64 ...
    ts_d_density            (shot_number, ts_radial_pos) float64 ...
    ts_d_temperature        (shot_number, ts_radial_pos) float64 ...
    adc_folder              (shot_number) object ...
    adc_calibration_index   (shot_number) object ...
    adc_4_probe             (shot_number) object ...
    adc_5_probe             (shot_number) object ...
    ts_temp_max             (shot_number) float64 ...
    ts_dens_max             (shot_number) float64 ...
    shot_end_time           (shot_number) datetime64[ns] ...
    ts_fl                   (shot_number) bool ...
    shot_time               (shot_number) datetime64[ns] ...
    shot_source_current     (shot_number) float64 ...
    shot_b_field            (shot_number) float64 ...
    shot_tilt               (shot_number) float64 ...
    shot_target_pos         (shot_number) float64 ...
    shot_target_rot         (shot_number) float64 ...
    shot_target_pumpspeed   (shot_number) int32 ...
    shot_source_pumpspeed   (shot_number) int32 ...
    shot_heating_pumpspeed  (shot_number) int32 ...
    shot_source_pos         (shot_number) float64 ...
    shot_deuterium_gf       (shot_number) float64 ...
    shot_hydrogen_gf        (shot_number) float64 ...
    shot_helium_gf          (shot_number) float64 ...
    shot_hydrogen_puffed    (shot_number) float64 ...
    shot_helium_puffed      (shot_number) float64 ...
    adc_timestamp           (shot_number) float64 ...
    adc_4_coax              (shot_number) float64 ...
    adc_4_shunt_resistance  (shot_number) float64 ...
    adc_5_coax              (shot_number) float64 ...
    adc_5_shunt_resistance  (shot_number) float64 ...
    adc_calibration_fl      (shot_number) float64 ...
    shot_fl                 (shot_number) float64 ...
    acquisition_length      (shot_number) float64 ...
    adc_end_time            (shot_number) datetime64[ns] ...
    adc_end_timestamp       (shot_number) float64 ...
    adc_freqs               (shot_number) float64 ...

In [6]:
meta_data_ds.set_coords('ts_dens_max')['ts_temp_max'].plot.line(x='ts_dens_max', linestyle='none', marker='x')

In [7]:
# Original 
b_field_markers = {
    0.8: 'o',
    1.2: 's',
    1.5: '^'
}
species_colours = {
    'Hydrogen': 'tab:blue',
    'Helium': 'tab:orange',
}

fig, ax = plt.subplots()


for species in ['Hydrogen', 'Helium']:
    for B_field in [0.8, 1.2, 1.5]:
        ds = meta_data_ds.where(np.logical_and(meta_data_ds.shot_b_field.round(1) == B_field, 
                                               meta_data_ds[f'shot_{species.lower()}_gf'].round(1) > 0.5))
        ds.set_coords('ts_dens_max')['ts_temp_max'].plot.line(x='ts_dens_max', linestyle='none', 
                                                              marker=b_field_markers[B_field], 
                                                              mfc='none', color=species_colours[species],
                                                              ax=ax, label=f'{B_field}T - {species}')
ax.set_xscale('log')
ax.set_xlabel(r'$n_{e, TS} \;$ [m$^{-3}$]')
ax.set_ylabel(r'$T_{e, TS} \;$ [eV]')
ax.legend()
# fig.tight_layout()

In [16]:
# Recolouring

b_field_markers = {
    0.8: 'o',
    1.2: 's',
    1.5: '^'
}
species_mfcs = {
    'Hydrogen': None,
    'Helium': 'none',
}
species_colours = {
    'Hydrogen': col.palettes['sb'],
    'Helium': col.palettes['sr'],
}

fig, ax = plt.subplots()


for species in ['Hydrogen', 'Helium']:
    for i, B_field in enumerate([0.8, 1.2, 1.5]):
        ds = meta_data_ds.where(np.logical_and(meta_data_ds.shot_b_field.round(1) == B_field, 
                                               meta_data_ds[f'shot_{species.lower()}_gf'].round(1) > 0.5))
        ds.set_coords('ts_dens_max')['ts_temp_max'].plot.line(x='ts_dens_max', linestyle='none', 
                                                              marker=b_field_markers[B_field], 
                                                              mfc=species_mfcs[species], 
                                                              color=species_colours[species][i+3],
                                                              ax=ax, label=f'{B_field}T - {species}')
ax.set_xscale('log')
ax.set_xlabel(r'$n_{e, TS} \;$ [m$^{-3}$]')
ax.set_ylabel(r'$T_{e, TS} \;$ [eV]')
ax.legend()
fig.tight_layout()